In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [2]:
ex1 = pd.read_parquet('train/4.청구입금정보/2018_청구정보_전체.parquet')

In [4]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '

In [5]:
cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']
print(ex1[cols].head())

  대표결제방법코드 대표청구지고객주소구분코드 대표청구서수령지구분코드 청구서수령방법 할인건수_R3M 할인건수_B0M
0     자동이체           미확인    당사페이앱+이메일   문자메세지    1회 이상    1회 이상
1     자동이체           주거지           우편      우편    1회 이상    1회 이상
2     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
3     자동이체           주거지           우편      우편    1회 이상    1회 이상
4     자동이체           주거지           우편      우편    1회 이상    1회 이상


In [6]:
print("할인건수_R3M unique values:")
print(ex1['할인건수_R3M'].unique())

print("\n할인건수_B0M unique values:")
print(ex1['할인건수_B0M'].unique())

할인건수_R3M unique values:
['1회 이상' '10회 이상' '20회 이상' '30회 이상' '40회 이상']

할인건수_B0M unique values:
['1회 이상' '10회 이상']


In [7]:
count_cols2 = ['할인건수_R3M', '할인건수_B0M']

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex1[col] = ex1[col].astype(str).str.replace('회 이상', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [8]:
le_cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes


In [9]:
ex1.head()

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,0,0,2,2,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,0,1,5,4,1,1,1,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,0,0,6,5,1,1,1,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,0,1,5,4,1,1,1,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,0,1,5,4,0,1,1,...,0,1,1,0,0,0,0,0,0,0


In [10]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_

In [11]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.001 ].index.tolist()
low_var

['대표결제방법코드']

In [12]:
cols_to_drop = ['대표결제방법코드']
ex1.drop(columns=cols_to_drop, inplace=True)

In [13]:
missing_mask = ex1.isna() | (ex1 == -1)

missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.3].index.tolist()

print("결측(또는 -1) 비율 > 30% 컬럼:", high_na)

결측(또는 -1) 비율 > 30% 컬럼: []


In [14]:
num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)

corr_matrix = num_df.corr().abs()
threshold = 0.7  # 상관계수 기준

high_corr_pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > threshold]

high_corr_feature_list = high_corr_pairs['Feature_2'].unique().tolist()

print("상관계수 초과 피처 목록:", high_corr_feature_list)


상관계수 초과 피처 목록: ['청구서수령방법', '청구서발송여부_R3M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '청구서발송여부_R6M', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '포인트_마일리지_환산_B0M', '포인트_포인트_건별_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_마일리지_월적립_R3M', '마일_잔여포인트_B0M', '포인트_포인트_월적립_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '혜택수혜금액_R3M', '혜택수혜금액', '선결제건수_R3M', '연체건수_R3M']


In [15]:
cols_to_drop = ['청구서수령방법', '청구서발송여부_R3M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '청구서발송여부_R6M', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '포인트_마일리지_환산_B0M', '포인트_포인트_건별_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_마일리지_월적립_R3M', '마일_잔여포인트_B0M', '포인트_포인트_월적립_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '혜택수혜금액_R3M', '혜택수혜금액', '선결제건수_R3M', '연체건수_R3M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [16]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 5]

print("VIF > 5인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 5인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [17]:
ex1

,기준년월,ID,대표결제일,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서발송여부_B0,청구금액_B0,포인트_마일리지_건별_B0M,포인트_포인트_건별_B0M,포인트_마일리지_월적립_B0M,포인트_포인트_월적립_B0M,포인트_잔여포인트_B0M,할인건수_R3M,할인금액_R3M,할인건수_B0M,선결제건수_R6M,연체건수_R6M
0,201807,TRAIN_000000,27,0,2,1,12226,0,1444,0,0,0,1,0,1,0,1
1,201807,TRAIN_000001,13,1,5,1,5834,0,0,0,0,0,1,0,1,0,0
2,201807,TRAIN_000002,1,0,6,1,21866,0,2305,0,0,0,1,0,1,0,0
3,201807,TRAIN_000003,5,1,5,1,16356,0,1452,0,0,0,1,0,1,2,0
4,201807,TRAIN_000004,13,1,5,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,25,1,5,0,0,0,0,0,0,0,1,0,1,0,0
2399996,201812,TRAIN_399996,20,0,6,1,14402,0,3339,0,0,0,1,0,1,0,0
2399997,201812,TRAIN_399997,20,2,5,1,5731,0,0,0,0,0,1,0,1,0,0
2399998,201812,TRAIN_399998,20,1,5,0,0,0,0,0,0,0,1,0,1,0,0


In [18]:
ex1.to_parquet('청구_전처리.parquet', index=False)

In [19]:
cols = ex1.columns.tolist()
cols

['기준년월',
 'ID',
 '대표결제일',
 '대표청구지고객주소구분코드',
 '대표청구서수령지구분코드',
 '청구서발송여부_B0',
 '청구금액_B0',
 '포인트_마일리지_건별_B0M',
 '포인트_포인트_건별_B0M',
 '포인트_마일리지_월적립_B0M',
 '포인트_포인트_월적립_B0M',
 '포인트_잔여포인트_B0M',
 '할인건수_R3M',
 '할인금액_R3M',
 '할인건수_B0M',
 '선결제건수_R6M',
 '연체건수_R6M']